LSTM

In [1]:
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 KB 716.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 KB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 KB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 KB 14.3 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.25.1
    Uninstalling requests-2.25.1:
      Successfully uninstalled requests-2.25.1
  Attempting uninstall: html5lib
    Found existing installation: html5lib 1.0.1
    Uninstalling html5lib-1.0.1:
      Successfully uninstalled html5lib-1.0.1
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
  

In [2]:
import pandas as pd
import yfinance as yf
import datetime
from datetime import date, timedelta
today = date.today()

In [3]:
d1 = today.strftime("%Y-%m-%d")
end_date = d1
d2 = date.today() - timedelta(days=5000)
d2 = d2.strftime("%Y-%m-%d")
start_date = d2

In [4]:
data = yf.download('BBCA.JK', 
                      start=start_date, 
                      end=end_date, 
                      progress=False)
data["Date"] = data.index
data = data[["Date", "Open", "High", "Low", "Close", "Adj Close", "Volume"]]
data.reset_index(drop=True, inplace=True)

In [5]:
data

,Date,Open,High,Low,Close,Adj Close,Volume
0,2009-05-13 00:00:00+07:00,650.0,665.0,635.0,650.0,526.791931,91475000
1,2009-05-14 00:00:00+07:00,650.0,650.0,630.0,650.0,526.791931,30230000
2,2009-05-15 00:00:00+07:00,650.0,660.0,640.0,650.0,526.791931,43685000
3,2009-05-18 00:00:00+07:00,670.0,685.0,635.0,670.0,543.001099,62312500
4,2009-05-19 00:00:00+07:00,700.0,700.0,680.0,700.0,567.314514,66137500
...,...,...,...,...,...,...,...
3391,2023-01-13 00:00:00+07:00,8150.0,8175.0,8025.0,8050.0,8050.000000,119748200
3392,2023-01-16 00:00:00+07:00,8050.0,8200.0,8050.0,8150.0,8150.000000,105710700
3393,2023-01-17 00:00:00+07:00,8175.0,8375.0,8150.0,8325.0,8325.000000,88695900
3394,2023-01-18 00:00:00+07:00,8200.0,8325.0,8175.0,8300.0,8300.000000,96814600


In [6]:
import plotly.graph_objects as go
figure = go.Figure(data=[go.Candlestick(x=data["Date"],
                                        open=data["Open"], 
                                        high=data["High"],
                                        low=data["Low"], 
                                        close=data["Close"])])
figure.update_layout(title = "Bank Central Asia Stock Price Analysis", 
                     xaxis_rangeslider_visible=False)
figure.show()

In [7]:
correlation = data.corr()
print(correlation["Close"].sort_values(ascending=False))

Close        1.000000
Low          0.999816
High         0.999795
Open         0.999659
Adj Close    0.999351
Volume       0.162772
Name: Close, dtype: float64


In [8]:
x = data[["Open", "High", "Low", "Volume"]]
y = data["Close"]
x = x.to_numpy()
y = y.to_numpy()
y = y.reshape(-1, 1)

from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(x, y, 
                                                test_size=0.25, 
                                                random_state=42)

In [9]:
from keras.models import Sequential
from keras.layers import Dense, LSTM
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape= (xtrain.shape[1], 1)))
model.add(LSTM(64, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 4, 128)            66560     
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 25)                1625      
                                                                 
 dense_1 (Dense)             (None, 1)                 26        
                                                                 
Total params: 117,619
Trainable params: 117,619
Non-trainable params: 0
_________________________________________________________________


In [18]:
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(xtrain, ytrain, batch_size=3, epochs=30)

Epoch 1/30
849/849 [==============================] - 9s 7ms/step - loss: 2994805.5000
Epoch 2/30
849/849 [==============================] - 6s 7ms/step - loss: 4658843.0000
Epoch 3/30
849/849 [==============================] - 6s 7ms/step - loss: 4320485.0000
Epoch 4/30
849/849 [==============================] - 6s 7ms/step - loss: 3356136.7500
Epoch 5/30
849/849 [==============================] - 7s 9ms/step - loss: 2101652.5000
Epoch 6/30
849/849 [==============================] - 7s 8ms/step - loss: 2835818.5000
Epoch 7/30
849/849 [==============================] - 6s 7ms/step - loss: 4966295.5000
Epoch 8/30
849/849 [==============================] - 6s 7ms/step - loss: 4932894.0000
Epoch 9/30
849/849 [==============================] - 6s 7ms/step - loss: 4925751.5000
Epoch 10/30
849/849 [==============================] - 6s 7ms/step - loss: 4925693.0000
Epoch 11/30
849/849 [==============================] - 6s 7ms/step - loss: 4925039.0000
Epoch 12/30
849/849 [====================

In [19]:
import numpy as np
#features = [Open, High, Low, Adj Close, Volume]
features = np.array([[8250.0, 8300.0, 8200.0, 66391800]])
model.predict(features)

1/1 [==============================] - 1s 828ms/step


array([[3672.09]], dtype=float32)